# Importing IDHP

In [1]:
import numpy as np
import pandas as pd
from dowhy import CausalModel

path_train = r"C:\Users\User\OneDrive\Desktop\bachelor_thesis\ihdp_npci_1-1000.train.npz"
path_test = r"C:\Users\User\OneDrive\Desktop\bachelor_thesis\ihdp_npci_1-1000.test.npz"
#path_train = r"C:\Users\User\OneDrive\Desktop\bachelor_thesis\ihdp_npci_1-100.train.npz"
#path_test = r"C:\Users\User\OneDrive\Desktop\bachelor_thesis\ihdp_npci_1-100.test.npz"



data_in_train = np.load(path_train)
data_in_test = np.load(path_test)
x = np.concatenate((data_in_train['x'], data_in_test['x']))
y = np.concatenate((data_in_train['yf'], data_in_test['yf']))
t = np.concatenate((data_in_train['t'], data_in_test['t']))



C:\Users\User\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### IHDP to multiple CSVs for R

In [ ]:
#for i in range(len(y[0])):
#    x_i = x[:,:,i]
#    y_i = y[:,i]
#    t_i = t[:,i]
#    col = ['t', 'y']
#    for j in range(1,26):
#        col.append("x"+str(j))
#    df_temp = pd.DataFrame(np.column_stack((t_i, y_i, x_i)), columns= col)
#    df_temp.to_csv(f"IHDP_data_CSVs/ihdp_{i+1}.csv", index= False)
#    

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, DotProduct
def average_std(std_arr):
    return 1 / len(std_arr) * np.sqrt(np.square(std_arr).sum()) 

def gp_ate_backdoor_binary_t(df_no_treatment, df_treatment, df_temp):
    kernel = 1.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3)) \
        + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
    gp_0= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=7,  alpha=0.0)
    gp_1= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=7,  alpha=0.0)

    # Fit the GP to the data
    gp_0.fit(df_no_treatment[["x"+str(i) for  i in range(1,26)]], df_no_treatment['y'])
    gp_1.fit(df_treatment [["x"+str(i) for  i in range(1,26)]], df_treatment['y'])

    gp_mu_1, gp_std_1 =  gp_1.predict(df_temp[["x"+str(i) for  i in range(1,26)]],return_std= True) 
    gp_mu_0, gp_std_0 =  gp_0.predict(df_temp[["x"+str(i) for  i in range(1,26)]],return_std= True)

    gp_ate = np.mean(gp_mu_1 - gp_mu_0)
    gp_ate_sd = average_std(gp_std_1) + average_std(gp_std_0) 
    return gp_ate, gp_ate_sd


def gp_linear_backdoor_binary_t(df_no_treatment, df_treatment, df_temp):
    kernel = DotProduct() + WhiteKernel()
    gp_0= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5,  alpha=0.0)
    gp_1= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5,  alpha=0.0)

    # Fit the GP to the data
    gp_0.fit(df_no_treatment[["x"+str(i) for  i in range(1,26)]], df_no_treatment['y'])
    gp_1.fit(df_treatment [["x"+str(i) for  i in range(1,26)]], df_treatment['y'])

    gp_mu_1, gp_std_1 =  gp_1.predict(df_temp[["x"+str(i) for  i in range(1,26)]],return_std= True) 
    gp_mu_0, gp_std_0 =  gp_0.predict(df_temp[["x"+str(i) for  i in range(1,26)]],return_std= True)

    gp_ate = np.mean(gp_mu_1 - gp_mu_0)
    gp_ate_sd = average_std(gp_std_1) + average_std(gp_std_0) 
    return gp_ate, gp_ate_sd

def gp_linear_backdoor_binary_t_v2(df_temp):
    kernel = DotProduct() + WhiteKernel()
    gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5,  alpha=0.0)
    

    # Fit the GP to the data
    col = ['t']+ ["x"+str(i) for  i in range(1,26)]
    gp.fit(df_temp[col ], df_temp['y'])
    x1 = np.array([1] + [0 for  i in range(1,26)])
    x0 = np.array([0 for  i in range(1,27)])
    gp_lin_ate =  gp.predict(x1.reshape(1,-1)) - gp.predict(x0.reshape(1,-1)) 
    
    return gp_lin_ate



In [9]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

def ipw_ate_backdoor_binary_t(df_temp):
    # Create a causal model from the data and given common causes.
    model=CausalModel(
            data = df_temp,
            treatment='t',
            outcome='y',
            common_causes=["x"+str(i) for  i in range(1,26)]
            )

    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True, method_name="maximal-adjustment")
    # Using Propensity Score Weighting
    estimate = model.estimate_effect(identified_estimand,
            method_name="backdoor.propensity_score_weighting"
    )
    return estimate.value


def ols_backdoor_binary_t(df_temp):
    # Create a causal model from the data and given common causes.
    model=CausalModel(
            data = df_temp,
            treatment='t',
            outcome='y',
            common_causes=["x"+str(i) for  i in range(1,26)]
            )

    identified_estimand = model.identify_effect(proceed_when_unidentifiable=True, method_name="maximal-adjustment")

    estimate = model.estimate_effect(identified_estimand,
            method_name="backdoor.linear_regression"
    )
    return estimate.value

def gp_linear_backdoor_binary_t_pymc(df_temp):
    import bambi as bmb
    import pymc as pm
    """
    Calculates the Markov Chain Monte Carlo trace of
    a Generalised Linear Model Bayesian linear regression 
    model on supplied data.
    Parameters
    ----------
    df: `pd.DataFrame`
        DataFrame containing the data
    iterations: `int`
        Number of iterations to carry out MCMC for
    """
    iterations=2500
    # Create the glm using the Bambi model syntax   
    bmb_string = "y ~ t + x1"
    for i in range(2,26):
        bmb_string += "+ x"+str(i)
    model = bmb.Model(bmb_string, df_temp)
    # Fit the model using a NUTS (No-U-Turn Sampler) 
    trace = model.fit(
        draws= iterations,
        tune=400,
        discard_tuned_samples=True,
        chains=1, 
        progressbar=True)
    return trace.posterior.t.to_numpy().mean()






In [19]:
from econml.dml import CausalForestDML

def cf_dml_backdoor_binary(df_temp):
    est = CausalForestDML( random_state=123)
    est.fit(Y = df_temp.y, T = df_temp.t, X = np.array(df_temp[["x"+str(i) for  i in range(1,26)]]), W =df_temp[["x"+str(i) for  i in range(1,26)]] )

    return est.ate(X = np.array(df_temp[["x"+str(i) for  i in range(1,26)]]))

In [ ]:
gp_ate_list= []
ipw_ate_list = []
ols_ate_list = []
tmle_ate_list = []
cf_dml_ate_list = []
gp_linear_ate_list = []


def get_ith_df(i, x, y, t):
    x_i = x[:,:,i]
    y_i = y[:,i]
    t_i = t[:,i]
    col = ['t', 'y']
    for j in range(1,26):
        col.append("x"+str(j))
    df_temp = pd.DataFrame(np.column_stack((t_i, y_i, x_i)), columns= col)
    return df_temp

for i in range(len(y[0])):
    
    df_temp = get_ith_df(i,  x, y, t)
    df_no_treatment = df_temp[df_temp.t == 0]
    df_treatment = df_temp[df_temp.t == 1]
    print(i+1,"/", len(y[0]))
    #GP RBF
    #gp_ate, gp_ate_sd = gp_ate_backdoor_binary_t(df_no_treatment, df_treatment, df_temp)
    #gp_ate_list.append(gp_ate)
    
    ### Causal Forest DML
    #cf_dml_ate_list.append(cf_dml_backdoor_binary(df_temp))

    ### IPW
    #ipw_ate_list.append(ipw_ate_backdoor_binary_t(df_temp))
#
    ### OLS
    #ols_ate_list.append(ols_backdoor_binary_t(df_temp))

    # GP linear
    gp_linear_ate_list.append(gp_linear_backdoor_binary_t_pymc(df_temp))
    
    
    

    
        
    



### Standard error of the mean: (ate +- sem(ate))

### Result for GP Gaussian kernel

In [31]:
import scipy
sem_gp_ate = scipy.stats.sem(np.array(gp_ate_list) - np.full((len(gp_ate_list),),4))

mean_error_gp_rbf =np.mean(np.array(gp_ate_list) - np.full((len(gp_ate_list),),4)) 
print(mean_error_gp_rbf, "+-", sem_gp_ate)

0.5538875379448825 +- 0.0591050715524681


### Result for Causal forest DML (n = 1000)

In [23]:
import scipy
sem_cf_dml_ate = scipy.stats.sem(np.array(cf_dml_ate_list) - np.full((len(cf_dml_ate_list),),4))

mean_error_cf_dml =np.abs(np.mean(np.array(cf_dml_ate_list) - np.full((len(cf_dml_ate_list),),4))) 
print(mean_error_cf_dml, "+-", sem_cf_dml_ate)

0.05097218389007281 +- 0.024999134444642947


### results for IPW

In [22]:
import scipy
sem_ipw_ate = scipy.stats.sem(np.array(ipw_ate_list) - np.full((len(ipw_ate_list),),4))

mean_error_ipw_ate =np.abs(np.mean(np.array(ipw_ate_list) - np.full((len(ipw_ate_list),),4))) 
print(mean_error_ipw_ate, "+-", sem_ipw_ate)

0.36501475076706896 +- 0.04109369390342119


### results for OLS

In [24]:
import scipy
sem_ols_ate = scipy.stats.sem(np.array(ols_ate_list) - np.full((len(ols_ate_list),),4))

mean_error_ols_ate =np.abs(np.mean(np.array(ols_ate_list) - np.full((len(ols_ate_list),),4))) 
print(mean_error_ols_ate, "+-", sem_ols_ate)

0.13326951863708864 +- 0.01411529524683612


# results for GP linear

In [27]:
import scipy
gp_ate_array_without_std = np.array(gp_linear_ate_list).reshape(1000)
sem_gp_linear_ate = scipy.stats.sem(np.array(gp_ate_array_without_std) - np.full((len(gp_ate_array_without_std),),4))

mean_error_gp_linear_ate =np.abs(np.mean(np.array(gp_ate_array_without_std) - np.full((len(gp_ate_array_without_std),),4))) 
print(mean_error_gp_linear_ate, "+-", sem_gp_linear_ate)

0.4940570461222967 +- 0.02317183858855958
